# Install Imports

In [1]:
import sys
!{sys.executable} -m pip install numpy scipy pillow scikit-image matplotlib
# works with python 3.7.3 numpy==1.15.4 scipy==1.5.4 scikit-image==0.16.1

# Import Modules

In [2]:
import numpy as np

import scipy.ndimage as ndimage

from os.path import join

from PIL import Image

from skimage import filters
from skimage import feature
from skimage.color import rgb2gray
from skimage.metrics import structural_similarity as ssim

import matplotlib.pyplot as plt

%matplotlib inline

# Define Metrics

Root Mean Squared Error   
(Wajid, R., Mansoor, A. B., & Pedersen, M. (2014, December). A human perception based performance evaluation of image quality metrics. In International Symposium on Visual Computing (pp. 303-312). Springer, Cham.)
  
Mean Squared Error   
(Wajid, R., Mansoor, A. B., & Pedersen, M. (2014, December). A human perception based performance evaluation of image quality metrics. In International Symposium on Visual Computing (pp. 303-312). Springer, Cham.)
  
Structural Similarity Index   
(Wajid, R., Mansoor, A. B., & Pedersen, M. (2014, December). A human perception based performance evaluation of image quality metrics. In International Symposium on Visual Computing (pp. 303-312). Springer, Cham.)

In [3]:
def rmse(src, dst):
    return np.sqrt(np.mean(np.square(src - dst)))


def mse(src, dst):
    return np.linalg.norm(src - dst)


def metric(src, dst):
    
    rms = rmse(src, dst)
    ms = mse(src, dst)
    sim = ssim(src, dst, multichannel=True)
    
    return rms, ms, sim

# Pooling-based time aware color smoothing



In [4]:
def running_pooling(matrix, neighbors, frames, steps=2, step_at_two=False):
    work_matrix = np.copy(matrix)
    return_matrix = np.copy(matrix)

    # Set step start
    step_idx = 1 if step_at_two else 0
    
    voting_matrix = [[1 if (i < neighbors / 2 and j <= (i + 1 - step_idx) * steps) or (i == int(neighbors / 2)) or (i > neighbors / 2 and j <= (neighbors - i - step_idx) * steps) else 0 for j in range(frames)] for i in range(neighbors)]
    voting_matrix = np.array(voting_matrix).astype('bool')
    
    # Append ones at top and bottom
    work_matrix = np.concatenate((np.ones((int(neighbors / 2), work_matrix.shape[1], work_matrix.shape[2])), work_matrix), axis=0)
    work_matrix = np.concatenate((work_matrix, np.ones((int(neighbors / 2), work_matrix.shape[1], work_matrix.shape[2]))), axis=0)
    
    # Append ones at end
    work_matrix = np.append(work_matrix, np.ones((work_matrix.shape[0], frames - 1, work_matrix.shape[2])), axis=1)
   
    for i in range(work_matrix.shape[1] - frames + 1):
        y_work_matrix = work_matrix[:,i:i + frames]
        for j in range(y_work_matrix.shape[0] - neighbors + 1):
            y_sub_work_matrix = y_work_matrix[j:j + neighbors]
            voted_matrix = y_sub_work_matrix[voting_matrix]
            voted_matrix = voted_matrix[voted_matrix[:,2].argsort()]
            voted_matrix = voted_matrix[voted_matrix[:,1].argsort(kind='mergesort')]
            voted_matrix = voted_matrix[voted_matrix[:,0].argsort(kind='mergesort')]
            value = np.median(voted_matrix, axis=0)
            return_matrix[j, i] = value
    
    return return_matrix

# Gaussian Blur

https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.gaussian_filter.html



In [5]:
def running_gaussian(matrix, sigma):
    return_matrix = np.copy(matrix)

    for i in range(im2arr.shape[1]):
        return_matrix[:,i] = ndimage.gaussian_filter(matrix[:,i], sigma=sigma, order=0)
        
    return return_matrix

# Load Spatial Rug

 - Load image data
 - Transform it to numpy array
 - Smooth by pooling
 - Smooth by Gaussian

In [6]:
rugs_path = 'rugs/'

im = Image.open(rugs_path + 'originalspatialrug.png')
im2arr = np.array(im)
arr2im = Image.fromarray(im2arr)

# Results

In [40]:
pooling_config = [
    [9, 9, 2],
    [11, 9, 2],
    [13, 9, 2],
    [15, 9, 2],
    [17, 9, 2],
    
    [9, 15, 2],
    [11, 15, 2],
    [13, 15, 2],
    [15, 15, 2],
    [17, 15, 2],
    
    [9, 17, 2],
    [11, 17, 2],
    [13, 17, 2],
    [15, 17, 2],
    [17, 17, 2],
    
    [9, 9, 9],
    [11, 11, 11],
    [13, 13, 13],
    [15, 15, 15],
    [17, 17, 17]
]

print('Amount of experiments', len(pooling_config))

pooling_results = []

for i, conf in enumerate(pooling_config):
    im2arr_neighbor = np.copy(im2arr)
    im2arr_neighbor = running_pooling(im2arr_neighbor, conf[0], conf[1], conf[2])
    metric_res = metric(im2arr, im2arr_neighbor)
    
    pooling_results.append([im2arr_neighbor, metric_res, conf])

    print('\rDone with experiment', i + 1, end=' ')

Amount of experiments 20
Done with experiment 20 

In [52]:
gaussian_config = [
    (1, 0),
    (2, 0),
    (3, 0),
    (4, 0),
    (5, 0),
    (6, 0),
    (7, 0),
    (8, 0),
    (9, 0)
]

gaussian_results = []

for conf in gaussian_config:
    im2arr_smooth = np.copy(im2arr)
    for i in range(im2arr.shape[1]):
        im2arr_smooth[:,i] = ndimage.gaussian_filter(im2arr[:,i], sigma=conf)
        
    metric_res = metric(im2arr, im2arr_smooth)
    gaussian_results.append([im2arr_smooth, metric_res, conf])

In [53]:
results_path = 'results'
edges_result_path = join(results_path, 'edges')
os.makedirs(edges_result_path, exist_ok=True)

grayscale = rgb2gray(im2arr)
edges = feature.canny(grayscale)

unique, counts = np.unique(edges, return_counts=True)
counts_matrix = np.asarray((unique, counts)).T
print(counts_matrix[0][1], counts_matrix[1][1], 'orig')

name = 'edges-orig-pixels-' + str(counts_matrix[1][1]) + '-'
name = join(edges_result_path, name) + '.png'

Image.fromarray(edges).save(name, 'PNG')

261792 40208 orig


In [51]:
for i, img in enumerate(pooling_results):
    grayscale = rgb2gray(img[0])
    edges = feature.canny(grayscale)

    unique, counts = np.unique(edges, return_counts=True)
    counts_matrix = np.asarray((unique, counts)).T
    print(counts_matrix[0][1], counts_matrix[1][1], img[2])

    name = 'edges-tacs-pixels-' + str(counts_matrix[1][1]) + '-' + '-'.join([str(x) for x in img[2]])
    name = join(edges_result_path, name) + '.png'
    
    Image.fromarray(edges).save(name, 'PNG')


281144 20856 [9, 9, 2]
285239 16761 [11, 9, 2]
288280 13720 [13, 9, 2]
290198 11802 [15, 9, 2]
291687 10313 [17, 9, 2]
280500 21500 [9, 15, 2]
283948 18052 [11, 15, 2]
286787 15213 [13, 15, 2]
288489 13511 [15, 15, 2]
289989 12011 [17, 15, 2]
280990 21010 [9, 17, 2]
283874 18126 [11, 17, 2]
286444 15556 [13, 17, 2]
288533 13467 [15, 17, 2]
289698 12302 [17, 17, 2]
285541 16459 [9, 9, 9]
288740 13260 [11, 11, 11]
290562 11438 [13, 13, 13]
292588 9412 [15, 15, 15]
293573 8427 [17, 17, 17]


In [54]:
for i, img in enumerate(gaussian_results):
    grayscale = rgb2gray(img[0])
    edges = feature.canny(grayscale)

    unique, counts = np.unique(edges, return_counts=True)
    counts_matrix = np.asarray((unique, counts)).T
    print(counts_matrix[0][1], counts_matrix[1][1], img[2])

    name = 'edges-gaussian-pixels-' + str(counts_matrix[1][1]) + '-' + '-'.join([str(x) for x in img[2]])
    name = join(edges_result_path, name) + '.png'
    
    Image.fromarray(edges).save(name, 'PNG')


271253 30747 (1, 0)
283658 18342 (2, 0)
292077 9923 (3, 0)
296288 5712 (4, 0)
298982 3018 (5, 0)
300758 1242 (6, 0)
301408 592 (7, 0)
301668 332 (8, 0)
301840 160 (9, 0)


In [55]:
im2 = Image.open(rugs_path + 'originalspatialrug-case3.png')
im2arr2 = np.array(im2)
arr2im2 = Image.fromarray(im2arr2)

In [56]:
grayscale = rgb2gray(im2arr2)
edges = feature.canny(grayscale)

unique, counts = np.unique(edges, return_counts=True)
counts_matrix = np.asarray((unique, counts)).T
print(counts_matrix[0][1], counts_matrix[1][1], 'orig')

name = 'edges-orig-case3-pixels-' + str(counts_matrix[1][1]) + '-'
name = join(edges_result_path, name) + '.png'

Image.fromarray(edges).save(name, 'PNG')

85683 3517 orig


In [60]:
for i, conf in enumerate(pooling_config):
    im2arr_neighbor = np.copy(im2arr2)
    im2arr_neighbor = running_pooling(im2arr_neighbor, conf[0], conf[1], conf[2])
    metric_res = metric(im2arr2, im2arr_neighbor)

    grayscale = rgb2gray(im2arr_neighbor)
    edges = feature.canny(grayscale)

    unique, counts = np.unique(edges, return_counts=True)
    counts_matrix = np.asarray((unique, counts)).T
    print(counts_matrix[0][1], counts_matrix[1][1], conf)

    name = 'edges-tacs-case3-pixels-' + str(counts_matrix[1][1]) + '-' + '-'.join([str(x) for x in conf])
    name = join(edges_result_path, name) + '.png'
    
    Image.fromarray(edges).save(name, 'PNG')

    name = 'true-tacs-case3-pixels-' + str(counts_matrix[1][1]) + '-' + '-'.join([str(x) for x in conf])
    name = join(edges_result_path, name) + '.png'
    
    Image.fromarray(im2arr_neighbor).save(name, 'PNG')

86128 3072 [9, 9, 2]
86208 2992 [11, 9, 2]
86438 2762 [13, 9, 2]
86957 2243 [15, 9, 2]
87110 2090 [17, 9, 2]
86197 3003 [9, 15, 2]
86431 2769 [11, 15, 2]
86782 2418 [13, 15, 2]
87204 1996 [15, 15, 2]
87123 2077 [17, 15, 2]
86237 2963 [9, 17, 2]
86577 2623 [11, 17, 2]
86775 2425 [13, 17, 2]
87256 1944 [15, 17, 2]
87104 2096 [17, 17, 2]
86284 2916 [9, 9, 9]
86521 2679 [11, 11, 11]
87039 2161 [13, 13, 13]
87195 2005 [15, 15, 15]
87308 1892 [17, 17, 17]


In [61]:
for conf in gaussian_config:
    im2arr_smooth = np.copy(im2arr2)
    for i in range(im2arr2.shape[1]):
        im2arr_smooth[:,i] = ndimage.gaussian_filter(im2arr2[:,i], sigma=conf)
    metric_res = metric(im2arr2, im2arr_smooth)

    grayscale = rgb2gray(im2arr_smooth)
    edges = feature.canny(grayscale)

    unique, counts = np.unique(edges, return_counts=True)
    counts_matrix = np.asarray((unique, counts)).T
    print(counts_matrix[0][1], counts_matrix[1][1], conf)

    name = 'edges-gaussian-case3-pixels-' + str(counts_matrix[1][1]) + '-' + '-'.join([str(x) for x in conf])
    name = join(edges_result_path, name) + '.png'
    
    Image.fromarray(edges).save(name, 'PNG')

    name = 'true-gaussian-case3-pixels-' + str(counts_matrix[1][1]) + '-' + '-'.join([str(x) for x in conf])
    name = join(edges_result_path, name) + '.png'
    
    Image.fromarray(im2arr_smooth).save(name, 'PNG')

86007 3193 (1, 0)
86686 2514 (2, 0)
87723 1477 (3, 0)
88090 1110 (4, 0)
88141 1059 (5, 0)
88277 923 (6, 0)
88260 940 (7, 0)
88219 981 (8, 0)
88291 909 (9, 0)
